In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive/')
  shared_drive_foldername = 'NUTRECON'
  root = '/content/drive/Shareddrives/{}/'.format(shared_drive_foldername)
  !pip install pandas==1.4.2
  print('Running Code in Colab')
# ...or locally
except:
  # define the local directory of the project
  root = 'D:/FV/Projects/NUTRECON/nutreconDrive/'
  print('Running Code locally')

import numpy as np
import sys
sys.path.append(root + 'python')
import nutrecon_simulation as sim

Running Code locally


## Task parameters

In [2]:
uniqueLott_Nreps= 6      # Unique Lottery Repititions  

# Same-type & mixed type Trials Lottery probabilities
st_refPs = [1]                              # Reference option
st_lottPs = [0.13, 0.22, 0.38, .50, .75]    # Lottery option

# Same-type task variables
st_money_refQs = [1]                               # Euros
st_money_lottQs = [1, 2, 5, 12, 20]                # Euros

st_cPlus_refQs = [20]                              # mL of CS+ yogurt 
st_cPlus_lottQs = [20, 40, 80, 120, 200]           # mL of CS+ yogurt

st_cMinus_refQs = st_cPlus_refQs                      # mL of CS- yogurt 
st_cMinus_lottQs = st_cPlus_lottQs                    # mL of CS- yogurt 

# Mixed-type task variables
mt_refQs = [.2]                                # Euros
mt_refPs = [1]
mt_lottPs = [0.13, 0.22, 0.38, .50, .75]

mt_cPlus_lottQs = [40, 80, 120, 150, 200]      # mL of CS+ yogurt 
mt_cMinus_lottQs = mt_cPlus_lottQs            # mL of CS- yogurt 


allTrials_df = sim.pack_taskParameters(
                st_refPs, st_lottPs, st_money_refQs, st_money_lottQs, st_cPlus_refQs, st_cPlus_lottQs,
                st_cMinus_refQs, st_cMinus_lottQs, mt_refQs, mt_refPs, mt_lottPs, mt_cPlus_lottQs,
                mt_cMinus_lottQs, uniqueLott_Nreps)

print('Trials per type:\n{}'.format(allTrials_df['trial_type'].value_counts()))

Trials per type:
same     450
mixed    300
Name: trial_type, dtype: int64


# Subject specific parameters

In [3]:
# %% Subject specific parameters
st_money_alpha = 0.7
st_cPlus_alpha = 0.8
st_cMinus_alpha = 0.8

st_money_beta = 1.8
st_cPlus_beta = st_money_beta
st_cMinus_beta = st_money_beta

mt_cPlus_beta = st_money_beta
mt_cMinus_beta = st_money_beta
cPlus_sFactor = .2
cMinus_sFactor = .2

subjectTrials_df = sim.pack_subjectParameters(st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, st_money_beta, 
                                              st_cPlus_beta, st_cMinus_beta, mt_cPlus_beta, mt_cMinus_beta, 
                                              cPlus_sFactor, cMinus_sFactor, allTrials_df)
subjectTrials_df


,trial_type,ref_type,ref_qt,ref_prob,lott_type,lott_qt,lott_prob,ref_alpha,lott_alpha,beta,sFactor
0,same,Money,1.0,1,Money,1,0.13,0.7,0.7,1.8,1.0
1,same,Money,1.0,1,Money,1,0.22,0.7,0.7,1.8,1.0
2,same,Money,1.0,1,Money,1,0.38,0.7,0.7,1.8,1.0
3,same,Money,1.0,1,Money,1,0.50,0.7,0.7,1.8,1.0
4,same,Money,1.0,1,Money,1,0.75,0.7,0.7,1.8,1.0
...,...,...,...,...,...,...,...,...,...,...,...
745,mixed,Money,0.2,1,CS-,200,0.13,0.7,0.8,1.8,0.2
746,mixed,Money,0.2,1,CS-,200,0.22,0.7,0.8,1.8,0.2
747,mixed,Money,0.2,1,CS-,200,0.38,0.7,0.8,1.8,0.2
748,mixed,Money,0.2,1,CS-,200,0.50,0.7,0.8,1.8,0.2


Simulate choices

In [4]:
seed = 1
np.random.seed(seed)

subjectTrials_df = sim.get_subject_choices(subjectTrials_df)
subjectTrials_df

,trial_type,ref_type,ref_qt,ref_prob,lott_type,lott_qt,lott_prob,ref_alpha,lott_alpha,beta,sFactor,ref_EU,lott_EU,pL,choice
0,same,Money,1.0,1,Money,1,0.13,0.7,0.7,1.8,1.0,1.000000,0.130000,0.172787,0
1,same,Money,1.0,1,Money,1,0.22,0.7,0.7,1.8,1.0,1.000000,0.220000,0.197182,0
2,same,Money,1.0,1,Money,1,0.38,0.7,0.7,1.8,1.0,1.000000,0.380000,0.246754,0
3,same,Money,1.0,1,Money,1,0.50,0.7,0.7,1.8,1.0,1.000000,0.500000,0.289050,0
4,same,Money,1.0,1,Money,1,0.75,0.7,0.7,1.8,1.0,1.000000,0.750000,0.389361,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,mixed,Money,0.2,1,CS-,200,0.13,0.7,0.8,1.8,0.2,0.324131,9.010883,0.934654,1
746,mixed,Money,0.2,1,CS-,200,0.22,0.7,0.8,1.8,0.2,0.324131,15.249187,0.992654,1
747,mixed,Money,0.2,1,CS-,200,0.38,0.7,0.8,1.8,0.2,0.324131,26.339504,0.999863,1
748,mixed,Money,0.2,1,CS-,200,0.50,0.7,0.8,1.8,0.2,0.324131,34.657242,0.999993,1


In [5]:
column_names = ['trial_type',
                'ref_type', 'ref_qt', 'ref_prob' ,
                'lott_type', 'lott_qt', 'lott_prob',
                'ref_alpha', 'lott_alpha', 'beta', 'sFactor',
                'ref_EU', 'lott_EU', 'pL', 'choice',
                'ref_alpha_est', 'lott_alpha_est', 'beta_est', 'sFactor_est',
                'ref_alpha_estStdErr', 'lott_alpha_estStdErr', 'beta_estStdErr']

optimize_cols = column_names[:7]  + column_names[14:19]
[print('{} -> {}'.format(p, optimize_cols[p])) for p in range(len(optimize_cols))];

0 -> trial_type
1 -> ref_type
2 -> ref_qt
3 -> ref_prob
4 -> lott_type
5 -> lott_qt
6 -> lott_prob
7 -> choice
8 -> ref_alpha_est
9 -> lott_alpha_est
10 -> beta_est
11 -> sFactor_est


In [6]:
# Checking if Likelihood calculation is working for simultaneous estimation and 10 parameters


df = subjectTrials_df.copy()
params = (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
          st_money_beta, 
        cPlus_sFactor, cMinus_sFactor)
df['likelihood'] = df.apply(lambda row: sim.get_likelihood(row, params), axis = 1)

tmp_df = df[['pL','choice', 'likelihood']].copy()

mask_0 = tmp_df['choice'] == 0
mask_1 = tmp_df['choice'] == 1

tmp_df.loc[mask_0,'check'] = 1 - tmp_df.loc[mask_0,'pL'] - tmp_df.loc[mask_0,'likelihood']
tmp_df.loc[mask_1,'check'] = tmp_df.loc[mask_1,'pL'] - tmp_df.loc[mask_1,'likelihood']
tmp_df['check'].max()

0.0

In [7]:
# Checking if Likelihood calculation is working for simultaneous estimation and 10 parameters


df = subjectTrials_df.copy()
params = (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
          st_money_beta, st_cPlus_beta, st_cMinus_beta, mt_cPlus_beta, mt_cMinus_beta, 
        cPlus_sFactor, cMinus_sFactor)
df['likelihood'] = df.apply(lambda row: sim.get_likelihood(row, params), axis = 1)

tmp_df = df[['pL','choice', 'likelihood']].copy()

mask_0 = tmp_df['choice'] == 0
mask_1 = tmp_df['choice'] == 1

tmp_df.loc[mask_0,'check'] = 1 - tmp_df.loc[mask_0,'pL'] - tmp_df.loc[mask_0,'likelihood']
tmp_df.loc[mask_1,'check'] = tmp_df.loc[mask_1,'pL'] - tmp_df.loc[mask_1,'likelihood']
tmp_df['check'].max()



0.0

In [8]:
# Checking if Likelihood calculation is working for stepwise estimation and 6 parameters


df = subjectTrials_df.copy()
params = (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
          st_money_beta, 
        cPlus_sFactor, cMinus_sFactor)
st_mask = df[column_names[0]] == 'same'
df.loc[st_mask,'likelihood'] = df.loc[st_mask,:].apply(lambda row: sim.get_st_likelihood(row, params), axis = 1)

mt_mask = df[column_names[0]] == 'mixed'
cPlus_mask = df[column_names[4]] == 'CS+'
cMinus_mask = df[column_names[4]] == 'CS-'

money_alpha_est = st_money_alpha
cPlus_alpha_est = st_cPlus_alpha
cMinus_alpha_est = st_cMinus_alpha
beta_est = st_money_beta

df.loc[mt_mask, optimize_cols[8]] = money_alpha_est
df.loc[cPlus_mask, optimize_cols[9]] = st_cPlus_alpha
df.loc[cMinus_mask, optimize_cols[9]] = cMinus_alpha_est
df.loc[mt_mask, optimize_cols[10]] = beta_est

df.loc[mt_mask,'likelihood'] = df.loc[mt_mask,:].apply(lambda row: sim.get_mt_likelihood(row, params), axis = 1)

tmp_df = df[['pL','choice', 'likelihood']].copy()

mask_0 = tmp_df['choice'] == 0
mask_1 = tmp_df['choice'] == 1

tmp_df.loc[mask_0,'check'] = 1 - tmp_df.loc[mask_0,'pL'] - tmp_df.loc[mask_0,'likelihood']
tmp_df.loc[mask_1,'check'] = tmp_df.loc[mask_1,'pL'] - tmp_df.loc[mask_1,'likelihood']
tmp_df['check'].max()

0.0

In [9]:
# Checking if Likelihood calculation is working for stepwise estimation and 10 parameters

df = subjectTrials_df.copy()
params = (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
          st_money_beta, st_cPlus_beta, st_cMinus_beta, mt_cPlus_beta, mt_cMinus_beta, 
        cPlus_sFactor, cMinus_sFactor)
st_mask = df[column_names[0]] == 'same'
df.loc[st_mask,'likelihood'] = df.loc[st_mask,:].apply(lambda row: sim.get_st_likelihood(row, params), axis = 1)

mt_mask = df[column_names[0]] == 'mixed'
cPlus_mask = df[column_names[4]] == 'CS+'
cMinus_mask = df[column_names[4]] == 'CS-'

money_alpha_est = st_money_alpha
cPlus_alpha_est = st_cPlus_alpha
cMinus_alpha_est = st_cMinus_alpha

df.loc[mt_mask, optimize_cols[8]] = money_alpha_est
df.loc[cPlus_mask, optimize_cols[9]] = st_cPlus_alpha
df.loc[cMinus_mask, optimize_cols[9]] = cMinus_alpha_est

df.loc[mt_mask,'likelihood'] = df.loc[mt_mask,:].apply(lambda row: sim.get_mt_likelihood(row, params), axis = 1)

tmp_df = df[['pL','choice', 'likelihood']].copy()

mask_0 = tmp_df['choice'] == 0
mask_1 = tmp_df['choice'] == 1

tmp_df.loc[mask_0,'check'] = 1 - tmp_df.loc[mask_0,'pL'] - tmp_df.loc[mask_0,'likelihood']
tmp_df.loc[mask_1,'check'] = tmp_df.loc[mask_1,'pL'] - tmp_df.loc[mask_1,'likelihood']
tmp_df['check'].max()

0.0

In [37]:
alphaMoney0 = 1
alphaCplus0 = 1
alphaCminus0 = 1
st_betaMoney0 = 1 # also used in model with only one beta
st_betaCplus0 = 1
st_betaCminus0 = 1
mt_betaCplus0 = 1
mt_betaCminus0 = 1
sFactorCplus0 = 1
sFactorCminus0= 1


x0_10params = (alphaMoney0, alphaCplus0, alphaCminus0, 
      st_betaMoney0, st_betaCplus0, st_betaCminus0, mt_betaCplus0, mt_betaCminus0,
      sFactorCplus0, sFactorCminus0)

x0_6params = (alphaMoney0, alphaCplus0, alphaCminus0, 
      st_betaMoney0,
      sFactorCplus0, sFactorCminus0)

In [ ]:
def print_simultaneousModel_output():
    print(50 * '=')
    print('{}\n  - parameters: {}\n  - std. error: {}'.format(res.message, res.x, np.sqrt(np.diag(res.hess_inv))))
    print('\nConfidene intervals:')
    parsCI = ['{} \xb1 {}'.format(round(res.x[p],3), round(1.96*np.sqrt(np.diag(res.hess_inv))[p],3)) for p in range(len(res.x))]
    for p in range(len(parsCI)): print('  - parameter {}: {}'.format(p + 1, parsCI[p]))
    
def print_stepwiseModel_output():
    print(50 * '=')
    print('{}\n  - parameters: {}\n  - std. error: {}'.format(res_st.message, res_st.x, np.sqrt(np.diag(res_st.hess_inv))))
    print('\nConfidene intervals:')
    parsCI = ['{} \xb1 {}'.format(round(res_st.x[p],3), round(1.96*np.sqrt(np.diag(res_st.hess_inv))[p],3)) for p in range(len(res_st.x))]
    for p in range(len(parsCI)): print('  - parameter {}: {}'.format(p + 1, parsCI[p]))
    print()
    print(50 * '=')
    print('{}\n  - parameters: {}\n  - std. error: {}'.format(res_mt.message, res_mt.x, np.sqrt(np.diag(res_mt.hess_inv))))
    print('\nConfidene intervals:')
    parsCI = ['{} \xb1 {}'.format(round(res_mt.x[p],3), round(1.96*np.sqrt(np.diag(res_mt.hess_inv))[p],3)) for p in range(len(res_mt.x))]
    for p in range(len(parsCI)): print('  - parameter {}: {}'.format(p + 1, parsCI[p]))

args = (subjectTrials_df.copy())
res_st, res_mt = sim.stepwise_estimate(args, x0_6params)



In [ ]:
print_stepwiseModel_output()

In [47]:
args = (subjectTrials_df.copy())
res_st, res_mt = sim.stepwise_estimate(args, x0_10params)

6

In [ ]:
print_stepwiseModel_output()

In [ ]:
res = sim.simultaneous_estimate(args, x0_6params)

In [ ]:
print_simultaneousModel_output()

In [ ]:
res = sim.simultaneous_estimate(args, x0_10params)

In [ ]:
print_simultaneousModel_output()